In [ ]:
import os
import re
import pandas as pd
import pickle
import collections
from collections import defaultdict
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open("ssg_brd_id_lst.txt", 'rb') as f:
    brd_id_lst = pickle.load(f)

In [ ]:
brd_w_prd = {}
exception = []

In [ ]:
for brd_id in tqdm(brd_id_lst):

    prd_id_lst = []
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    socket.setdefaulttimeout(30)

    try:
        wd.get('http://www.ssg.com/disp/brandShop.ssg?brandId={}'.format(brd_id))
        time.sleep(3)

        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        try:
            last_page_num = int(re.findall('changePage\((\d*?)\)', str(soup.find("a", {"class": "btn_last on"})))[0])
        except:
            last_page_num = 1

        for page in range(1, last_page_num+1):
            wd.get('http://www.ssg.com/disp/brandShop.ssg?brandId={}&page={}'.format(brd_id, page))
            time.sleep(3)
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            
            cunit_t290 = soup.find_all("li", {"class": "cunit_t290"})
            
            if cunit_t290 != []:
                for i in cunit_t290:
                    prd_clickable = i.find_all("div", {"class": "title"})
                    prd_id_lst.append(int(re.findall('data-info="(\d*?)"', str(prd_clickable))[0]))

        brd_w_prd[brd_id] = prd_id_lst
        
    except Exception as e:
        exception.append(brd_id)
        print("에러가 발생했습니다.")
        print(brd_id, ": ", e)
        pass